# 기초부터 시작하는 NLP: SEQUENCE TO SEQUENCE 네트워크와 ATTENTION을 이용한 번역
![](https://tutorials.pytorch.kr/_images/seq2seq.png)  
https://arxiv.org/abs/1409.0473  
# 데이터 파일 로딩
이 프로젝트의 데이터는 수천 개의 한국어-영어 번역 쌍입니다.

Open Data Stack Exchange 에 관한 이 질문은 https://tatoeba.org/eng/downloads 에서 다운 로드가 가능한 공개 번역 사이트 https://tatoeba.org/ 를 알려 주었습니다. 더 나은 방법으로 언어 쌍을 개별 텍스트 파일로 분할하는 추가 작업을 수행한 https://www.manythings.org/anki/ 가 있습니다:

In [1]:
from __future__ import unicode_literals, print_function, division
from io import open
import unicodedata
import string
import re
import random

import torch
import torch.nn as nn
from torch import optim
import torch.nn.functional as F

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

c:\Users\kimju\OneDrive\바탕 화면\Study\파이썬\machine learning\pytorch_practice\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
SOS_token = 0
EOS_token = 1


class Lang:
    def __init__(self, name):
        self.name = name
        self.word2index = {}
        self.word2count = {}
        self.index2word = {0: "SOS", 1: "EOS"}
        self.n_words = 2  # SOS 와 EOS 포함

    def addSentence(self, sentence):
        for word in sentence.split(' '):
            self.addWord(word)

    def addWord(self, word):
        if word not in self.word2index:
            self.word2index[word] = self.n_words
            self.word2count[word] = 1
            self.index2word[self.n_words] = word
            self.n_words += 1
        else:
            self.word2count[word] += 1

In [3]:
# 유니 코드 문자열을 일반 ASCII로 변환하십시오.
# https://stackoverflow.com/a/518232/2809427
def unicodeToAscii(s):
    return ''.join(
        c for c in unicodedata.normalize('NFD', s)
        if unicodedata.category(c) != 'Mn'
    )

# 소문자, 다듬기, 그리고 문자가 아닌 문자 제거
def normalizeString(s):
    s = unicodeToAscii(s.lower().strip())
    s = re.sub(r"([.!?])", r" \1", s)
    s = re.sub(r"/[^a-z|A-Z|0-9|ㄱ-ㅎ|가-힣]/g", r" ", s)
    return s

#### 데이터 파일을 읽기 위해 파일을 줄로 나눈 다음 줄을 쌍으로 나눈다.
#### 파일은 모두 영어 → 기타 언어이므로 다른 언어 → 영어에서 번역하려면 플래그를 추가하여 reverse 쌍을 뒤집는다

In [4]:
def readLangs(lang1, lang2, reverse=False):
    print("Reading lines...")

    # 파일을 읽고 줄로 분리
    temp = open('../data/eng-kor.txt', encoding='utf-8').\
        read().strip().split('\n')
    
    lines=[]
    for i in temp:
        t=i.split('\t')
        lines.append(t[1]+'\t'+t[3])
        
    # 모든 줄을 쌍으로 분리하고 정규화
    pairs = [[normalizeString(s) for s in l.split('\t')] for l in lines]

    # 쌍을 뒤집고, Lang 인스턴스 생성
    pairs = [list(p) for p in pairs]
    input_lang = Lang(lang1)
    output_lang = Lang(lang2)
    
    return input_lang, output_lang, pairs


In [5]:
MAX_LENGTH = 64


def filterPair(p):
    return len(p[0].split(' ')) < MAX_LENGTH and \
        len(p[1].split(' ')) < MAX_LENGTH


def filterPairs(pairs):
    return [pair for pair in pairs if filterPair(pair)]

In [6]:
def prepareData(lang1, lang2, reverse=False):
    input_lang, output_lang, pairs = readLangs(lang1, lang2, reverse)
    print("Read %s sentence pairs" % len(pairs))
    pairs = filterPairs(pairs)
    print("Trimmed to %s sentence pairs" % len(pairs))
    print("Counting words...")
    for pair in pairs:
        input_lang.addSentence(pair[0])
        output_lang.addSentence(pair[1])
    print("Counted words:")
    print(input_lang.name, input_lang.n_words)
    print(output_lang.name, output_lang.n_words)
    return input_lang, output_lang, pairs


input_lang, output_lang, pairs = prepareData('kor', 'eng', True)
print(random.choice(pairs))

Reading lines...
Read 8595 sentence pairs
Trimmed to 8594 sentence pairs
Counting words...
Counted words:
kor 5112
eng 12088
['cut me some slack .', '나 좀 내버려 둬 .']


# Seq2Seq 모델  
-   모든 입력에 해당하는 출력이 있는 단일 RNN의 시퀀스 예측과 달리 Seq2Seq 모델은 시퀀스 길이와 순서를 자유롭게하기 때문에 두 언어 사이의 번역에 이상적입니다.  
-   입력 단어의 시퀀스를 직역해서 정확한 번역을 만드는 것은 어려울 것입니다.  
-   Seq2Seq 모델을 사용하면 인코더는 하나의 벡터를 생성합니다. 이상적인 경우에 입력 시퀀스의 《의미》를 문장의 N 차원 공간에 있는 단일 지점인 단일 벡터으로 인코딩합니다.  
![](https://tutorials.pytorch.kr/_images/seq2seq.png)  

# 인코더
-   Seq2Seq 네트워크의 인코더는 입력 문장의 모든 단어에 대해 어떤 값을 출력하는 RNN
-   모든 입력 단어에 대해 인코더는 벡터와 은닉 상태를 출력하고 다음 입력 단어를 위해 그 은닉 상태를 사용  
![](https://tutorials.pytorch.kr/_images/encoder-network.png)

In [7]:
class EncoderRNN(nn.Module):
    def __init__(self, input_size, hidden_size):
        super(EncoderRNN, self).__init__()
        self.hidden_size = hidden_size

        self.embedding = nn.Embedding(input_size, hidden_size)
        self.gru = nn.GRU(hidden_size, hidden_size)

    def forward(self, input, hidden):
        embedded = self.embedding(input).view(1, 1, -1)
        output = embedded
        output, hidden = self.gru(output, hidden)
        return output, hidden

    def initHidden(self):
        return torch.zeros(1, 1, self.hidden_size, device=device)

# 디코더
-   디코더는 인코더 출력 벡터를 받아서 번역을 생성하기 위한 단어 시퀀스를 출력

## 간단한 디코더
-   가장 간단한 seq2seq 디코더는 인코더의 마지막 출력만을 이용
-   이 마지막 출력은 전체 시퀀스에서 문맥을 인코드하기 때문에 문맥 벡터(context vector)로 불린다.
-   문맥 벡터는 디코더의 초기 은닉 상태로 사용
-   디코딩의 매 단계에서 디코더에게 입력 토큰과 은닉 상태가 주어진다.
-   초기 입력 토큰은 SOS(start-of-string)토큰이고 첫 은닉 상태는 문맥 벡터(인코더의 마지막 은닉 상태)이다.  
![](https://tutorials.pytorch.kr/_images/decoder-network.png)

In [8]:
class DecoderRNN(nn.Module):
    def __init__(self, hidden_size, output_size):
        super(DecoderRNN, self).__init__()
        self.hidden_size = hidden_size

        self.embedding = nn.Embedding(output_size, hidden_size)
        self.gru = nn.GRU(hidden_size, hidden_size)
        self.out = nn.Linear(hidden_size, output_size)
        self.softmax = nn.LogSoftmax(dim=1)

    def forward(self, input, hidden):
        output = self.embedding(input).view(1, 1, -1)
        output = F.relu(output)
        output, hidden = self.gru(output, hidden)
        output = self.softmax(self.out(output[0]))
        return output, hidden

    def initHidden(self):
        return torch.zeros(1, 1, self.hidden_size, device=device)

# Attention 디코더
-   문맥 벡터만 인코더와 디코더 사이로 전달 된다면, 단일 벡터가 전체 문장을 인코딩 해야하는 부담을 가지게 된다.
-   Attention은 디코더 신경망이 자기 출력의 모든 단계에서 인코더 출력의 다른 부분에 집중 할 수 있게 한다.
-   첫째 Attention 가중치의 합을 계산
-   이것은 가중치 조합을 만들기 위해서 인코더 출력 벡터와 곱해진다.
-   그 결과 입력 시퀀스의 특정 부분에 관한 정보를 포함해야하고 따라서 디코더가 알맞은 출력 단어를 선택하는 것을 도와준다.  
![](https://i.imgur.com/1152PYf.png)

-   어텐션 가중치 계산은 디코더의 입력 및 은닉 상태를 입력으로 사용하는 다른 feed-forward 계층인 attn으로 수행
-   학습 데이터에는 모든 크기의 문장이 있기 때문에 적용할 수 있는 최대 문장 길이를 선택해야 한다.  
![](https://tutorials.pytorch.kr/_images/attention-decoder-network.png)

In [9]:
class AttnDecoderRNN(nn.Module):
    def __init__(self, hidden_size, output_size, dropout_p=0.1, max_length=MAX_LENGTH):
        super(AttnDecoderRNN, self).__init__()
        self.hidden_size = hidden_size
        self.output_size = output_size
        self.dropout_p = dropout_p
        self.max_length = max_length

        self.embedding = nn.Embedding(self.output_size, self.hidden_size)
        self.attn = nn.Linear(self.hidden_size * 2, self.max_length)
        self.attn_combine = nn.Linear(self.hidden_size * 2, self.hidden_size)
        self.dropout = nn.Dropout(self.dropout_p)
        self.gru = nn.GRU(self.hidden_size, self.hidden_size)
        self.out = nn.Linear(self.hidden_size, self.output_size)

    def forward(self, input, hidden, encoder_outputs):
        embedded = self.embedding(input).view(1, 1, -1)
        embedded = self.dropout(embedded)

        attn_weights = F.softmax(
            self.attn(torch.cat((embedded[0], hidden[0]), 1)), dim=1)
        attn_applied = torch.bmm(attn_weights.unsqueeze(0),
                                 encoder_outputs.unsqueeze(0))

        output = torch.cat((embedded[0], attn_applied[0]), 1)
        output = self.attn_combine(output).unsqueeze(0)

        output = F.relu(output)
        output, hidden = self.gru(output, hidden)

        output = F.log_softmax(self.out(output[0]), dim=1)
        return output, hidden, attn_weights

    def initHidden(self):
        return torch.zeros(1, 1, self.hidden_size, device=device)

# 학습
## 학습데이터 준비
-   학습을 위해 각 쌍마다 입력 Tensor와 목표 Tensor가 필요하다.
-   이 벡터들을 생성하는 동안 두 시퀀스에 EOS토큰을 추가


In [10]:
def indexesFromSentence(lang, sentence):
    return [lang.word2index[word] for word in sentence.split(' ')]


def tensorFromSentence(lang, sentence):
    indexes = indexesFromSentence(lang, sentence)
    indexes.append(EOS_token)
    return torch.tensor(indexes, dtype=torch.long, device=device).view(-1, 1)


def tensorsFromPair(pair):
    input_tensor = tensorFromSentence(input_lang, pair[0])
    target_tensor = tensorFromSentence(output_lang, pair[1])
    return (input_tensor, target_tensor)

# 모델 학습
-   학습을 위해서 인코더에 입력 문장을 넣고 모든 출력과 최신 은닉 상태를 추적
-   디코더에 첫 번째 입력으로 <sos> 토큰과 인코더의 마지막 은닉 상태가 첫번째 은닉 상태로 제공됨


In [11]:
teacher_forcing_ratio = 0.5


def train(input_tensor, target_tensor, encoder, decoder, encoder_optimizer, decoder_optimizer, criterion, max_length=MAX_LENGTH):
    encoder_hidden = encoder.initHidden()

    encoder_optimizer.zero_grad()
    decoder_optimizer.zero_grad()

    input_length = input_tensor.size(0)
    target_length = target_tensor.size(0)

    encoder_outputs = torch.zeros(max_length, encoder.hidden_size, device=device)

    loss = 0

    for ei in range(input_length):
        encoder_output, encoder_hidden = encoder(
            input_tensor[ei], encoder_hidden)
        encoder_outputs[ei] = encoder_output[0, 0]

    decoder_input = torch.tensor([[SOS_token]], device=device)

    decoder_hidden = encoder_hidden

    use_teacher_forcing = True if random.random() < teacher_forcing_ratio else False

    if use_teacher_forcing:
        # Teacher forcing 포함: 목표를 다음 입력으로 전달
        for di in range(target_length):
            decoder_output, decoder_hidden, decoder_attention = decoder(
                decoder_input, decoder_hidden, encoder_outputs)
            loss += criterion(decoder_output, target_tensor[di])
            decoder_input = target_tensor[di]  # Teacher forcing

    else:
        # Teacher forcing 미포함: 자신의 예측을 다음 입력으로 사용
        for di in range(target_length):
            decoder_output, decoder_hidden, decoder_attention = decoder(
                decoder_input, decoder_hidden, encoder_outputs)
            topv, topi = decoder_output.topk(1)
            decoder_input = topi.squeeze().detach()  # 입력으로 사용할 부분을 히스토리에서 분리

            loss += criterion(decoder_output, target_tensor[di])
            if decoder_input.item() == EOS_token:
                break

    loss.backward()

    encoder_optimizer.step()
    decoder_optimizer.step()

    return loss.item() / target_length

In [12]:
import time
import math


def asMinutes(s):
    m = math.floor(s / 60)
    s -= m * 60
    return '%dm %ds' % (m, s)


def timeSince(since, percent):
    now = time.time()
    s = now - since
    es = s / (percent)
    rs = es - s
    return '%s (- %s)' % (asMinutes(s), asMinutes(rs))

In [13]:
def trainIters(encoder, decoder, n_iters, print_every=1000, plot_every=100, learning_rate=0.01):
    start = time.time()
    plot_losses = []
    print_loss_total = 0  # print_every 마다 초기화
    plot_loss_total = 0  # plot_every 마다 초기화

    encoder_optimizer = optim.SGD(encoder.parameters(), lr=learning_rate)
    decoder_optimizer = optim.SGD(decoder.parameters(), lr=learning_rate)
    training_pairs = [tensorsFromPair(random.choice(pairs))
                      for i in range(n_iters)]
    criterion = nn.NLLLoss()

    for iter in range(1, n_iters + 1):
        training_pair = training_pairs[iter - 1]
        input_tensor = training_pair[0]
        target_tensor = training_pair[1]

        loss = train(input_tensor, target_tensor, encoder,
                     decoder, encoder_optimizer, decoder_optimizer, criterion)
        print_loss_total += loss
        plot_loss_total += loss

        if iter % print_every == 0:
            print_loss_avg = print_loss_total / print_every
            print_loss_total = 0
            print('%s (%d %d%%) %.4f' % (timeSince(start, iter / n_iters),
                                         iter, iter / n_iters * 100, print_loss_avg))

        if iter % plot_every == 0:
            plot_loss_avg = plot_loss_total / plot_every
            plot_losses.append(plot_loss_avg)
            plot_loss_total = 0

    showPlot(plot_losses)

# 결과 도식화

In [14]:
import matplotlib.pyplot as plt
plt.switch_backend('agg')
import matplotlib.ticker as ticker
import numpy as np


def showPlot(points):
    plt.figure()
    fig, ax = plt.subplots()
    # 주기적인 간격에 이 locator가 tick을 설정
    loc = ticker.MultipleLocator(base=0.2)
    ax.yaxis.set_major_locator(loc)
    plt.plot(points)

In [15]:
# 평가
def evaluate(encoder, decoder, sentence, max_length=MAX_LENGTH):
    with torch.no_grad():
        input_tensor = tensorFromSentence(input_lang, sentence)
        input_length = input_tensor.size()[0]
        encoder_hidden = encoder.initHidden()

        encoder_outputs = torch.zeros(max_length, encoder.hidden_size, device=device)

        for ei in range(input_length):
            encoder_output, encoder_hidden = encoder(input_tensor[ei],
                                                     encoder_hidden)
            encoder_outputs[ei] += encoder_output[0, 0]

        decoder_input = torch.tensor([[SOS_token]], device=device)  # SOS

        decoder_hidden = encoder_hidden

        decoded_words = []
        decoder_attentions = torch.zeros(max_length, max_length)

        for di in range(max_length):
            decoder_output, decoder_hidden, decoder_attention = decoder(
                decoder_input, decoder_hidden, encoder_outputs)
            decoder_attentions[di] = decoder_attention.data
            topv, topi = decoder_output.data.topk(1)
            if topi.item() == EOS_token:
                decoded_words.append('<EOS>')
                break
            else:
                decoded_words.append(output_lang.index2word[topi.item()])

            decoder_input = topi.squeeze().detach()

        return decoded_words, decoder_attentions[:di + 1]

In [16]:
def evaluateRandomly(encoder, decoder, n=10):
    for i in range(n):
        pair = random.choice(pairs)
        print('>', pair[0])
        print('=', pair[1])
        output_words, attentions = evaluate(encoder, decoder, pair[0])
        output_sentence = ' '.join(output_words)
        print('<', output_sentence)
        print('')

In [17]:
hidden_size = 256
encoder1 = EncoderRNN(input_lang.n_words, hidden_size).to(device)
attn_decoder1 = AttnDecoderRNN(hidden_size, output_lang.n_words, dropout_p=0.1).to(device)

trainIters(encoder1, attn_decoder1, 75000, print_every=5000)

0m 44s (- 10m 18s) (5000 6%) 5.3935
1m 21s (- 8m 50s) (10000 13%) 5.1767
1m 59s (- 7m 57s) (15000 20%) 4.8260
2m 37s (- 7m 12s) (20000 26%) 4.5337
3m 15s (- 6m 30s) (25000 33%) 4.1985
3m 53s (- 5m 50s) (30000 40%) 3.8144
4m 31s (- 5m 10s) (35000 46%) 3.5183
5m 9s (- 4m 31s) (40000 53%) 3.2174
5m 47s (- 3m 51s) (45000 60%) 2.9074
6m 26s (- 3m 13s) (50000 66%) 2.6713
7m 5s (- 2m 34s) (55000 73%) 2.4063
7m 43s (- 1m 55s) (60000 80%) 2.1364
8m 22s (- 1m 17s) (65000 86%) 1.9772
9m 1s (- 0m 38s) (70000 93%) 1.7291
9m 40s (- 0m 0s) (75000 100%) 1.5676


In [18]:
def evaluateRandomly(encoder, decoder, n=10):
    for i in range(n):
        pair = random.choice(pairs)
        print('>', pair[0])
        print('=', pair[1])
        output_words, attentions = evaluate(encoder, decoder, pair[0])
        output_sentence = ' '.join(output_words)
        print('<', output_sentence)
        print('')

In [30]:
output_words, attentions = evaluate(encoder1, attn_decoder1, 'Fucking')
output_sentence = ' '.join(output_words)
print(output_sentence)

KeyError: 'Fucking'

In [20]:
evaluateRandomly(encoder1, attn_decoder1)

> i have time to do that this week .
= 나는 이번주에 저걸 할 시간이 있어 .
< 나는 이번주에 저걸 할 시간이 있어 . <EOS>

> i volunteered .
= 나는 자원했다 .
< 나는 자원했다 . <EOS>

> it's fucking hot .
= 존나 더워 .
< 존나 더워 . <EOS>

> you lied to me, didn't you ?
= 너 나 속였지 ?
< 너 나 속였지 ? <EOS>

> the nail tore his jacket .
= 못이 그의 자켓을 찢었다 .
< 그의 그의 자켓을 찢었다 . <EOS>

> layla seemed sleepy .
= 라일라는 졸려 보였어 .
< 라일라는 졸려 보였어 . <EOS>

> tom tried to think logically .
= 톰은 논리적으로 생각하려 애썼다 .
< 톰은 논리적으로 생각하려 애썼다 . <EOS>

> are you able to swim ?
= 수영하실 수 있나요 ?
< 수영하실 수 있나요 ? <EOS>

> seriously ?
= 진심이야 ?
< 나 ? ? <EOS>

> this sounds familiar .
= 귀에 익은 소리인걸 .
< 귀에 익은 소리인걸 . <EOS>

